In [1]:
# Based on
# https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.2-understanding-recurrent-neural-networks.ipynb

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.8.0


In [5]:
# https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification
max_features = 10000  # number of words to consider as features
maxlen = 500  # cut texts after this number of words (among top max_features most common words)

# each review is encoded as a sequence of word indexes
# indexed by overall frequency in the dataset
# output is 0 (negative) or 1 (positive) 
imdb = tf.keras.datasets.imdb.load_data(num_words=max_features)
(raw_input_train, y_train), (raw_input_test, y_test) = imdb

In [6]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

input_train = tf.keras.preprocessing.sequence.pad_sequences(raw_input_train, maxlen=maxlen)
input_test = tf.keras.preprocessing.sequence.pad_sequences(raw_input_test, maxlen=maxlen)

In [7]:
input_train.shape, input_test.shape, y_train.shape, y_test.shape

((25000, 500), (25000, 500), (25000,), (25000,))

In [8]:
# tf.keras.layers.SimpleRNN?
# tf.keras.layers.Embedding?

In [9]:
embedding_dim = 32

model = tf.keras.Sequential()
# Parameters: max_features (10000) * 8 = 80000 
model.add(tf.keras.layers.Embedding(name='embedding', input_dim=max_features, output_dim=embedding_dim, input_length=maxlen))

# model.add(tf.keras.layers.Embedding(max_features, 32, input_length=maxlen))
# model.add(tf.keras.layers.SimpleRNN(32, return_sequences=True))
# model.add(tf.keras.layers.SimpleRNN(32, return_sequences=True))

# https://arxiv.org/ftp/arxiv/papers/1701/1701.05923.pdf
# n = output dimension
# m = input dimension
# Total number of parameters for 
# RNN = n**2 + nm (like fc) + n (bias)

# n = 1, m =32: 1 + 32 + 1 = 34
# model.add(tf.keras.layers.SimpleRNN(name='rnn', units=1))

# n = 32, m =32: 1024 + 1024 + 32 = 2080
model.add(tf.keras.layers.SimpleRNN(name='rnn1', units=32, return_sequences=True))
model.add(tf.keras.layers.SimpleRNN(name='rnn2', units=32))

# Input format: maxlen (500) * dimension of embedding (8)
# Output: 4000
# model.add(tf.keras.layers.Flatten())

# binary classifier
model.add(tf.keras.layers.Dense(name='fc', units=32, activation='relu'))
model.add(tf.keras.layers.Dense(name='classifier', units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           320000    
_________________________________________________________________
rnn1 (SimpleRNN)             (None, 500, 32)           2080      
_________________________________________________________________
rnn2 (SimpleRNN)             (None, 32)                2080      
_________________________________________________________________
fc (Dense)                   (None, 32)                1056      
_________________________________________________________________
classifier (Dense)           (None, 1)                 33        
Total params: 325,249
Trainable params: 325,249
Non-trainable params: 0
_________________________________________________________________


In [10]:
batch_size = 1000

%time history = model.fit(input_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 10s 507us/step - loss: 0.6947 - acc: 0.5122 - val_loss: 0.6915 - val_acc: 0.5282
Epoch 2/10
20000/20000 [==============================] - 7s 343us/step - loss: 0.6878 - acc: 0.5428 - val_loss: 0.6894 - val_acc: 0.5304
Epoch 3/10
20000/20000 [==============================] - 7s 341us/step - loss: 0.6792 - acc: 0.5720 - val_loss: 0.6835 - val_acc: 0.5562
Epoch 4/10
20000/20000 [==============================] - 7s 353us/step - loss: 0.6641 - acc: 0.5993 - val_loss: 0.6733 - val_acc: 0.5818
Epoch 5/10
20000/20000 [==============================] - 7s 344us/step - loss: 0.6364 - acc: 0.6427 - val_loss: 0.6474 - val_acc: 0.6256
Epoch 6/10
20000/20000 [==============================] - 7s 364us/step - loss: 0.5840 - acc: 0.6947 - val_loss: 0.6042 - val_acc: 0.6798
Epoch 7/10
20000/20000 [==============================] - 8s 379us/step - loss: 0.5364 - acc: 0.7327 - val_loss: 0.5853 - 

In [11]:
train_loss, train_accuracy = model.evaluate(input_train, y_train, batch_size=batch_size)
train_accuracy

25000/25000 [==============================] - 2s 84us/step


0.8506800007820129

In [12]:
test_loss, test_accuracy = model.evaluate(input_test, y_test, batch_size=batch_size)
test_accuracy

25000/25000 [==============================] - 2s 84us/step


0.7861999988555908

In [13]:
# precition
model.predict(input_test[0:5])

array([[0.09507405],
       [0.9440881 ],
       [0.4846489 ],
       [0.77605283],
       [0.9677393 ]], dtype=float32)

In [14]:
# ground truth
y_test[0:5]

array([0, 1, 1, 0, 1], dtype=int64)